# Setup Environment

In [ ]:
# Setup configuration
import sys
sys.path.append('/Users/benyoung/projects/ai-me')

from src.config import Config
from IPython.display import Markdown
from agents import trace, Runner, Agent, Tool

config = Config()

print(config)

# Download, Load, Chunk, Vectorize and Store md files in Chroma

In [ ]:
from importlib import reload
import src.data as _data_module
reload(_data_module)
from src.data import DataManager


# Use consolidated data manager
# For some reason, the glob pattern does not follow symlinks properly, so specify directly here
data_manager = DataManager(
    doc_load_local=["me/**/*.md"],
    github_repos=config.github_repos
)

# Load all repos configured in config.github_repos (automatically loads based on list presence)
chunks = data_manager.load_and_process_all(github_repos=config.github_repos)

print(f"Total chunks created: {len(chunks)}")

# Create the vectorstore using DataManager
vectorstore = data_manager.create_vectorstore(chunks, reset=True)
retriever = vectorstore.as_retriever()


data_manager.show_docs_for_file("faq.md")


# Setup Agents

In [ ]:
from src.agent import AgentConfig

# Initialize agent config with vectorstore
agent_config = AgentConfig(
    bot_full_name=config.bot_full_name, 
    github_token=config.github_token.get_secret_value(),
    model=config.model,
    vectorstore=vectorstore
)

ai_me = await agent_config.create_ai_me_agent()


# Run Agent Tests

In [ ]:
# Reload agent module to pick up latest changes
import src.agent as _agent_module
reload(_agent_module)
from src.agent import AgentConfig

# Recreate agent config with updated module
agent_config = AgentConfig(
    bot_full_name=config.bot_full_name, 
    github_token=config.github_token.get_secret_value(),
    model=config.model,
    vectorstore=vectorstore
)

# Create agent with custom prompt
ai_me = await agent_config.create_ai_me_agent(
    agent_prompt="""
You are acting as somebody who personifying {self.bot_full_name} and must follow these rules:
 * If the user asks a question, use the get_local_info tool to gather more info
 * Answer based on the information given to you by the tool calls
 * do not offer follow up questions, just answer the question
 * Add Reference-style links if they contain https://github.com
 * Get the current time if the user asks a time relative question
 * Only search the following list of GitHub repos Neosofia/corporate, byoung/me, byoung/ai-me
  """)

with trace("test-1"):
    result = await Runner.run(ai_me, "What do you know about ReaR?")

display(Markdown(result.final_output))


In [ ]:
with trace("test-2"):
    result = await Runner.run(ai_me, "What do you know about python?")
display(Markdown(result.final_output))


In [ ]:
with trace("test-3"):
    result = await Runner.run(ai_me, "Give me a summary of all the commits you've made in the last week")
display(Markdown(result.final_output))


In [ ]:
with trace("test-4"):
    result = await Runner.run(ai_me, "who is slartibartfast?")
display(Markdown(result.final_output))

In [ ]:
import gradio

async def chat(user_input: str, history):
    print("================== USER ===================")
    print(user_input)

    result = await Runner.run(ai_me, user_input)

    print("================== AGENT ==================")
    print(result.final_output)
    return result.final_output

with gradio.Blocks(theme=gradio.themes.Ocean()) as ui:

    gradio.Markdown(f"""# Welcome to {config.app_name}
                    The digital version of {config.bot_full_name}
                    The digital assistant that you never knew you needed ;)
                    Feel free to ask me anything about my experience, skills, projects, and interests.
                    """)
    gradio.ChatInterface(chat, type="messages")

ui.launch()

# The End